# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless') # 창 숨기기


In [11]:
# from user_agent import generate_user_agent, generate_navigator

# print(generate_user_agent(device_type='desktop'))
# print(generate_user_agent(os='win', device_type='desktop'))

# navigator = generate_navigator()

# random_user = generate_user_agent(os='win', device_type='desktop')
# print(random_user)

In [2]:
from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = options)

#driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


In [3]:
url_df = pd.read_csv("./나무문화/2022/나무문화_링크_2022_9월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(987, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/plumtreelee-/222888764192,2022 정조대왕 능행차 공동재현(서울 구간 축제소개/일정...,2022.09.30.
1,https://blog.naver.com/kimjihye525/222888763332,[전주야경] 덕진공원 & 카페한올 - 가을밤 산책코스,2022.09.30.
2,https://blog.naver.com/iland2279/222888762836,가을의 문지기 9월의 아라반,2022.09.30.
3,https://blog.naver.com/aceyun88/222888760839,"정남진 장흥 여행 특산물, 귀족호도박물관,귀족호도를 아시나요?",2022.09.30.
4,https://blog.naver.com/numain64/222888760244,"살기 싫어질때 가는 여행, 순천으로 가세요 (시즌 2)",2022.09.30.
...,...,...,...
982,https://blog.naver.com/ka1186/222864429353,노원관광 이야기 - 서울 생활사 박물관 별관 (구치감동),2022.09.01.
983,https://blog.naver.com/ge0307/222864429290,[애정맛집] 카페 - 커피를 찾아서 (광교웰빙국민체육센터),2022.09.01.
984,https://blog.naver.com/grant2/222864428566,"여주 가볼만한곳 강천섬 고목의 고혹함, 남한강...",2022.09.01.
985,https://blog.naver.com/tear911/222864426193,대구팔공산맛집 신숭겸유적지 옆 화덕촌 대구지묘점!,2022.09.01.


In [4]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


987


In [7]:
df = pd.DataFrame(columns = ['url','본문']) # 처음 만들때를 위한 코드라 주석처리 하였음.
df.to_csv("./나무문화/2022/나무문화_본문_2022_9월.csv", index=False, encoding='UTF-8')
df.to_excel("./나무문화/2022/나무문화_본문_2022_9월.xlsx", index=False, encoding='UTF-8')

c:\Users\user\anaconda3\envs\NIFS\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [9]:
df = pd.read_csv("./나무문화/2022/나무문화_본문_2022_9월.csv", encoding='UTF-8')
df

,url,본문


In [10]:
len(df)

0

In [18]:
url_df.iloc[11354]

url      https://blog.naver.com/dltnwls5397/222077716819
title                   [향수] 프로소디 런던 PROSODY LONDON. com
date                                         2020.09.02.
Name: 11354, dtype: object

In [19]:
df.isnull().sum()

본문     4
url    0
dtype: int64

In [20]:
# 이건 필요할때 쓰면 됨. 블로그가 닫혔거나 삭제되었을때 빈 본문을 가진 데이터프레임 추가.
# content_dic = {'url' : ["https://blog.naver.com/fbi8864/221998085418"], '본문' : [None]}
# additional_df = pd.DataFrame(content_dic)
# df = df.append(additional_df, ignore_index = True)

In [11]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []
waitsec = random.uniform(4,4.5)

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(10)
    time.sleep(waitsec)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
        # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("./나무문화/2022/나무문화_본문_2022_9월.csv", index=False, encoding='UTF-8')
    #df.to_excel("./나무문화/2022/나무문화_본문_2022_9월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    
    display.clear_output(wait=True) # 출력지우기
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break


986 986


C:\Users\user\AppData\Local\Temp\ipykernel_13108\3810532493.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)


In [ ]:
df

In [ ]:
print(len(contents))
#print(len(old_blogs))
print(len(hide_link))
